In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Importing heart disease dataset from github
data = pd.read_csv("/content/drive/MyDrive/diabetes.csv")


# Selecting Input and Output data
X = data.drop('Outcome', axis=1)
y = data['Outcome']


# Printing the number of missing values
print("Number of missing values:")
print(X.isnull().sum())


# Spliting the data into training and test sets:
# Assuming X is your features and y is your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Printing the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Number of missing values:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64
Shape of X_train: (614, 8)
Shape of X_test: (154, 8)
Shape of y_train: (614,)
Shape of y_test: (154,)


## Particle Swarm Optimization algorithm (PSO )

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import time

class Particle:
    def __init__(self, dim):
        self.position = np.random.rand(dim)
        self.velocity = np.random.rand(dim)
        self.best_position = self.position
        self.best_fitness = 0

def objective_function(hyperparameters):

    # Extract hyperparameters
    C, gamma = hyperparameters

    if(C < 0):
      C *= -1
    if(gamma < 0):
      gamma *= -1

    # Create an SVM model with the given hyperparameters
    model = SVC(C=C, gamma=gamma)

    #Training the model
    model.fit(X_train, y_train)

    # Evaluate the model using accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    return accuracy




def update_velocity(particle, global_best_position, inertia_weight=0.9, c1=0.5, c2=0.3):
    inertia_term = inertia_weight * particle.velocity
    cognitive_term = c1 * np.random.rand() * (particle.best_position - particle.position)
    social_term = c2 * np.random.rand() * (global_best_position - particle.position)
    particle.velocity = inertia_term + cognitive_term + social_term




def pso_classification(num_particles, dim, max_iter):
    particles = [Particle(dim) for _ in range(num_particles)]
    global_best_position = None
    global_best_accuracy = float('-inf')

    for _ in range(max_iter):
        for particle in particles:
            accuracy = objective_function(particle.position)

            if accuracy > particle.best_fitness:
                particle.best_fitness = accuracy
                particle.best_position = particle.position

            if accuracy > global_best_accuracy:
                global_best_accuracy = accuracy
                global_best_position = particle.position

        for particle in particles:
            update_velocity(particle, global_best_position)
            particle.position += particle.velocity

    return global_best_position, global_best_accuracy


if __name__ == "__main__":

    num_particles = 30
    dim = 2
    max_iter = 100

    start_time = time.time()
    best_position, best_accuracy = pso_classification(num_particles, dim, max_iter)
    pso_time = time.time() - start_time


    print("PSO Time:", pso_time)
    print("Best Accuracy: ", best_accuracy)

PSO Time: 102.69166398048401
Best Accuracy:  0.6428571428571429


## Ant colony optimization algorithm (ACO)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC



# Ant Colony Optimization-inspired feature selection
class AntColony:
    def __init__(self, num_ants, num_iterations, num_params, q0=0.9, beta=1, rho=0.1):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.num_params = num_params
        self.q0 = q0
        self.beta = beta
        self.rho = rho

    def run(self):
        #initializing an array filled with ones
        pheromones = np.ones((self.num_params))
        best_features = None
        best_accuracy = 0.0
        ant_features = self.generate_ant_features(self.num_params)

        for _ in range(self.num_iterations):
            ant_accuracy = self.evaluate_ant(ant_features)

            if ant_accuracy > best_accuracy:
                best_accuracy = ant_accuracy
                best_features = ant_features

            # Update pheromones
            pheromones *= (1 - self.rho)


        return best_features, best_accuracy


    def generate_ant_features(self, num_params):
        # Assume each feature is represented by an ant
        #initialize an array
        ant_features = np.random.choice([0.001, 10], size=num_params)
        return ant_features


    def evaluate_ant(self, ant_features):

        C = ant_features[0]
        gamma = ant_features[1]
        # Train a classifier and evaluate accuracy
        model = SVC(kernel='linear')

        #Training the model
        model.fit(X_train, y_train)

        # Evaluate the model using accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy




if __name__ == "__main__":

  start_time = time.time()
  # Create an instance of AntColonyFeatureSelection
  aco_fs = AntColony(num_ants=10, num_iterations=100, num_params=2)

  # Run ACO-inspired feature selection
  best_features, best_accuracy = aco_fs.run()
  aco_time = time.time() - start_time

  # Display the results
  print("ACO Time:", aco_time)
  print("Best Accuracy: ", best_accuracy)

ACO Time: 350.55983567237854
Best Accuracy:  0.7532467532467533


## Conclusion

 ACO demonstrated better accuracy, it came at the cost of increased computational time compared to PSO.


1.  ACO took 350.56 units of time and achieved a best accuracy of approximately 75.32%
2.  PSO took 102.69 units of time and achieved a best accuracy of around 64.29%.

